<a href="https://colab.research.google.com/github/YJonmo/StockMarket/blob/main/StockMarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yfinance
!pip install torch torchvision torchaudio
!pip install opencv-python


     |████████████████████████████████| 5.5MB 11.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=240a91b05f7d23b771a09ab18ddd0e8b3d396cbe8a1dfaff86f3ba303052325d
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 1.9MB 8.2MB/s 


In [ ]:
#This code uploads a list of stock's names and finds their variances and gains. 
#It also applies Least Squared Error algorithm to predict the stock prices and 
# returns the angles and the biases, and residual error. Using these info it is 
# possible to find the stocks which are more predictable.

import numpy as np
import yfinance as yf
from google.colab import files
from google.colab import drive
from matplotlib import pyplot as plt
import torch
Stocks = []

#drive.mount('/content/drive')
#uploaded = files.upload()
file = open('/content/drive/My Drive/Colab Notebooks/StocksonStake _ All.txt', 'r')

NAZDAQ_names = file.readlines() 
NAZDAQ_names3 = []

for i in range(len(NAZDAQ_names)):
#for i in range(20):
    print("\r"+"Reading stock data from " + NAZDAQ_names[i].split(' ')[0], end = "")
    Stock = yf.Ticker(NAZDAQ_names[i].split(' ')[0])
    
    #old = Stock.history(start="2021-01-08")
    old = Stock.history(period="2mo")
    if not(old.empty):
#        file2.write(NAZDAQ_names[i])
        
        old.head()
        old = old.reset_index()
        #for i in ['Open', 'High', 'Close', 'Low']: 
        #old["Close"] = old["Close"].astype('float64')
        Stocks.append(old["Close"].astype('float64'))
        NAZDAQ_names3.append(NAZDAQ_names[i])

Reading stock data from ACIC.U- ACIC.U: No data found, symbol may be delisted
Reading stock data from AKO.A- AKO.A: No data found, symbol may be delisted
Reading stock data from BF.A- BF.A: No data found, symbol may be delisted
Reading stock data from BF.B- BF.B: No data found for this date range, symbol may be delisted
Reading stock data from BHVN

In [16]:
print(len(Stocks))
Stocks2 = []
NAZDAQ_names2 = []
Length = len(Stocks[0])
for i in range(len(Stocks)):
    if len(Stocks[i]) == Length:
        Stocks2.append(Stocks[i])
        NAZDAQ_names2.append(NAZDAQ_names3[i])


import torch

Stocks2 = np.array(Stocks2)
Stocks_Flatten = np.array(Stocks2)
X = np.linspace(0,Length-1, Length)
Mean = np.zeros((len(Stocks2)), np.float64)
Var = np.zeros((len(Stocks2)), np.float64)
Gain = np.zeros((len(Stocks2)), np.float64)
Angle = np.zeros((len(Stocks2)), np.float64)
Residu = np.zeros((len(Stocks2)), np.float64)

for i in range(len(Stocks2)):
    Stocks_Flatten[i] = Stocks2[i]/np.max(Stocks2[i])*100
    #Gain[i] = Stocks2[i][Length-1] - Stocks2[i][0]
    #Coeffs = X*Gain[i]/(Length -1 + 0.00001)
    #Stocks_Flatten[i] = Stocks2[i]-Coeffs
    Stocks_Flatten[i] = (Stocks_Flatten[i]-np.min(Stocks_Flatten[i]))
    Stocks_Flatten[i] = 100*Stocks_Flatten[i]/(np.max(Stocks_Flatten[i]))
    Gain[i] = Stocks_Flatten[i][Length-1] - Stocks_Flatten[i][0]
    Coeffs = (X*Gain[i])/(Length -1 + 0.00001)
    Stocks_Flatten[i] = Stocks_Flatten[i]-Coeffs
    Var[i] = np.var(Stocks_Flatten[i])   
    Mean[i] = np.mean(Stocks_Flatten[i])    

    
    A = np.vstack([X, np.ones(Length)]).T
    model_parameters, residuals, rank, singular_values = np.linalg.lstsq(A, Stocks2[i], rcond=None)
    #m, c = np.linalg.lstsq(A, Stocks2[i], rcond=None)[0]
    Angle[i],_ = model_parameters
    #Residu[i] = residuals
    Yh = np.dot(A, model_parameters)
    Residu[i] = np.square(Stocks2[i] - Yh).mean()

3862


In [17]:
Threshold_above = 0.10
Threshold_below = 0.9
Gainer_indexes = np.where((Angle>Threshold_above)&(Threshold_below>Angle))
Gain  = Gain[Gainer_indexes]
Angle = Angle[Gainer_indexes]
Residu= Residu[Gainer_indexes]
Var   = Var[Gainer_indexes]
Stocks_Flatten = Stocks_Flatten[Gainer_indexes]
Stocks2 = Stocks2[Gainer_indexes]
NAZDAQ_names = []
for i in Gainer_indexes[0]:
  NAZDAQ_names.append(NAZDAQ_names2[i])

In [2]:
print(len(NAZDAQ_names))
i = np.argmin(Residu)           # the criteria for choosing a stock
#print(i)
model_parameters, residuals, rank, singular_values = np.linalg.lstsq(A, Stocks2[i], rcond=None)
Yh = np.dot(A, model_parameters)
plt.plot(np.transpose([(Stocks2[i]), Yh]))
print("Stock name: " + NAZDAQ_names[i])
#print(np.arctan(Angle[i])*57.2)


NameError: ignored

In [ ]:
print(len(Stocks))
Stocks2 = torch.tensor(())
NAZDAQ_names2 = []
Length = len(Stocks[0])
for i in range(len(Stocks)):
    if len(Stocks[i]) == Length:
        #Stocks2.append(Stocks[i])
        NAZDAQ_names2.append(NAZDAQ_names3[i])
        Stack = torch.tensor([Stocks[i]]).float()
        Stocks2 = torch.cat((Stocks2, Stack), 0)

Stocks_Flatten = Stocks2.detach().clone()
X     = torch.linspace(0,Length-1, steps=1)
Mean  = torch.zeros(len(Stocks2))
Var   = torch.zeros(len(Stocks2))
Gain  = torch.zeros(len(Stocks2))
Angle = torch.zeros(len(Stocks2))
Residu= torch.zeros(len(Stocks2))
print((Stocks2.shape))
print(Gain[0])
print(torch.mean(Stocks2.data[10]))